# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [4]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [ ]:
pd.read_sql("""
        SELECT firstName, lastName
            FROM employees
        JOIN offices
            USING(officeCode)
        WHERE city = "Boston";
""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [27]:
offices = pd.read_sql("""
        SELECT officeCode AS office_code, city, COUNT() AS number_of_employees
            FROM employees e
        JOIN offices 
        USING(officeCode)
        GROUP BY officeCode;
""", conn)

offices


,office_code,city,number_of_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [41]:
"""
How many customers are there per office?
"""

pd.read_sql("""
        SELECT officeCode AS office_code, city, COUNT(customerNumber) AS number_of_customers
            FROM employees e
        LEFT JOIN customers c 
        ON e.employeeNumber = c.salesRepEmployeeNumber
        GROUP BY officeCode;
""", conn)

,office_code,city,number_of_customers
0,1,None,12
1,2,Cambridge,12
2,3,NYC,15
3,4,None,29
4,5,Singapore,5
5,6,None,10
6,7,Helsinki,17


In [42]:
"""
Which city has the highest number of employees?
"""

pd.read_sql("""
        SELECT officeCode AS office_code, city, COUNT(officeCode) AS number_of_employees
            FROM employees e
        LEFT JOIN offices 
        USING(officeCode)
        GROUP BY officeCode
        ORDER BY officeCode
       ;
""", conn)

,office_code,city,number_of_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


In [ ]:
"""
Which city has fewest customers?
"""

pd.read_sql("""
        SELECT officeCode AS office_code, city, COUNT(customerNumber) AS number_of_customers
            FROM employees e
        LEFT JOIN customers c 
        ON e.employeeNumber = c.salesRepEmployeeNumber
        GROUP BY officeCode
        ORDER BY number_of_customers
        LIMIT 1;
""", conn)

# Your code here

,office_code,city,number_of_customers
0,5,Singapore,5


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [47]:
pd.read_sql("""
        SELECT productName AS products_sold
            FROM orderdetails od
        JOIN products p
        USING(productCode)
        JOIN orders o
        USING(orderNumber)
        LIMIT 30
;
""", conn)

,products_sold
0,1917 Grand Touring Sedan
1,1911 Ford Town Car
2,1932 Alfa Romeo 8C2300 Spider Sport
3,1936 Mercedes Benz 500k Roadster
4,1932 Model A Ford J-Coupe
5,1928 Mercedes-Benz SSK
6,1939 Chevrolet Deluxe Coupe
7,1938 Cadillac V-16 Presidential Limousine
8,1937 Lincoln Berline
9,1936 Mercedes-Benz 500K Special Roadster


In [57]:
pd.read_sql("""
        SELECT firstName, lastName, productName
        FROM employees AS e
        JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders
            USING(customerNumber)
        JOIN orderdetails
            USING(orderNumber)
        JOIN products
            USING(productCode)
;
""", conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [59]:
pd.read_sql("""
        SELECT firstName, lastName, SUM(quantityOrdered) as total_products_sold
        FROM employees AS e
        JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders
            USING(customerNumber)
        JOIN orderdetails
            USING(orderNumber)
        GROUP BY firstName, lastName
        ORDER BY total_products_sold
;
""", conn)

,firstName,lastName,total_products_sold
0,Leslie,Thompson,4056
1,Martin,Gerard,4180
2,Julie,Firrelli,4227
3,Mami,Nishi,4923
4,Foon Yue,Tseng,5016
5,Steve,Patterson,5561
6,Loui,Bondur,6186
7,Andy,Fixter,6246
8,Peter,Marsh,6632
9,George,Vanauf,7423


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [63]:
pd.read_sql("""
        SELECT firstName, lastName, COUNT(productCode) as different_products_sold
        FROM employees AS e
        JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
        JOIN orders
            USING(customerNumber)
        JOIN orderdetails
            USING(orderNumber)
        GROUP BY firstName, lastName
        HAVING different_products_sold > 200
        ORDER BY lastName
;
""", conn)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!